In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient
import json
import pandas as pd

In [2]:
AZURE_FORM_RECOGNIZER_ENDPOINT = "https://form-service-rtw.cognitiveservices.azure.com/"
AZURE_FORM_RECOGNIZER_KEY = "6752434521604f128f1c6f62236050df"

endpoint = AZURE_FORM_RECOGNIZER_ENDPOINT
key = AZURE_FORM_RECOGNIZER_KEY

form_recognizer_client = FormRecognizerClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [3]:
id_list = [
    "libby-herold",
    "sameer-kumar",
    "james-jackson",
    "james-webb",
    "radha-s-kumar",
    "avkash-chauhan"
]

In [4]:
def get_id_card_details(identity_card):
    first_name = identity_card.fields.get("FirstName").value
    last_name = identity_card.fields.get("LastName").value
    dob = identity_card.fields.get("DateOfBirth").value.strftime("%-m/%-d/%y")
    
    return first_name, last_name, dob

In [5]:
flight_manifest = pd.read_csv('../material_preparation_step/flight-manifest-csv/FlightManifest.csv')

In [6]:
for id_name in id_list:
    content_url = f"https://github.com/RyanWhitcomb-VT/automated_passenger_boarding_kiosk/blob/main/material_preparation_step/digital_id_template/ca-dl-{id_name}.png?raw=true"

    id_content_from_url = form_recognizer_client.begin_recognize_identity_documents_from_url(content_url)
    collected_id_cards = id_content_from_url.result()
        
    pred_fn, pred_ln, pred_dob = get_id_card_details(collected_id_cards[0])
    
    temp_df = flight_manifest[(flight_manifest['First Name'] == pred_fn) & (flight_manifest['Last Name'] == pred_ln) & (flight_manifest['DateofBirth'] == pred_dob)]
    if not temp_df.empty:
        print(f"{id_name} First name, last name, and date of birth all validated with flight manifest")
    else:
        print(pred_fn, pred_ln, pred_dob)
    

libby-herold First name, last name, and date of birth all validated with flight manifest
sameer-kumar First name, last name, and date of birth all validated with flight manifest
james-jackson First name, last name, and date of birth all validated with flight manifest
james-webb First name, last name, and date of birth all validated with flight manifest
Radha SKumar 3/5/94
AVKASH CHAUHAN CHAUHAN 1/1/90


Radha failed validation due to the lack of a space between the middle initial and last name
Avkash failed validation due to the capitalization of his name that was unexpected in the manifest